In [4]:
import pandas as pd
import emoji


mistral = pd.read_csv("../LLMs/Mixtral/responsesMixtral.csv")
mistral.head()

,Prompt,Response
0,Generate one tweet similar to the following on...,"""Excited for the start of #COP26 in Glasgow 🏴..."
1,Generate one tweet similar to the following on...,"""#cdnpoli #cdnmedia: Imagining a media game-c..."
2,Generate one tweet similar to the following on...,"""Reflecting on the silver lining of the pande..."
3,Generate one tweet similar to the following on...,"""Corruption personified: Greedy, dishonest, s..."
4,Generate one tweet similar to the following on...,"""@CNBC @GretaThunberg: Irony alerts as politi..."


In [5]:
#delete Emoji
def delete_emojis(text):
    return emoji.demojize(text)


mistral["Generated Response"] = mistral["Generated Response"].apply(delete_emojis)
mistral["Generated Response"] = mistral["Generated Response"].apply(delete_emojis)


In [6]:
#remove all the quotes from the columns
mistral["Generated Response"] = mistral["Generated Response"].str.replace('"', "").replace("'", "")
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace('"', "").replace("'", "")

In [7]:
#remove urls from both columns
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [8]:
#remove every tagging of a user
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'@\S+', '', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'@\S+', '', regex=True)

In [9]:
#remove every newline character
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'\n', ' ', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'\n', ' ', regex=True)

In [10]:
#remove every tab character
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'\t', ' ', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'\t', ' ', regex=True)

In [11]:
#remove every other special character to be sure
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'[^\w\s]', '', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'[^\w\s]', '', regex=True)

In [12]:
#remove every number
mistral["Generated Response"] = mistral["Generated Response"].str.replace(r'\d+', '', regex=True)
mistral["Original Tweet"] = mistral["Original Tweet"].str.replace(r'\d+', '', regex=True)

In [13]:
#all to lower case
mistral["Generated Response"] = mistral["Generated Response"].str.lower()
mistral["Original Tweet"] = mistral["Original Tweet"].str.lower()

In [15]:
#remove stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

mistral["Generated Response"] = mistral["Generated Response"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
mistral["Original Tweet"] = mistral["Original Tweet"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
#lemmatize everything
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.tag import pos_tag

# Download necessary ntkl data files
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]
    return ' '.join(lemmatized_tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
mistral["Generated Response"] = mistral["Generated Response"].apply(lemmatize_text)
mistral["Original Tweet"] = mistral["Original Tweet"].apply(lemmatize_text)

In [18]:
mistral.to_csv("llama3csv.csv")